In [ ]:
import pandas as pd
import os

In [ ]:
ROOT = "Data"
IMAGES = f"{ROOT}/Train_Images"
LABELS = f"{ROOT}/labels"
DATA = f"{ROOT}/Train.csv"
YOLO_DATA = f"{ROOT}/Transformed_Data"
IMAGE_SIZE = 512

In [ ]:
classes = {
    "fruit_woodiness": 0,
    "fruit_brownspot": 1,
    "fruit_healthy": 2
}

In [ ]:
df = pd.read_csv(DATA)
image_filenames = os.listdir(IMAGES)

In [ ]:
def get_metric_data_from_image(image_df):
    x_center = image_df["xmin"] + image_df["width"]/2
    y_center = image_df["ymin"] + image_df["height"]/2
    data = [x_center, y_center, image_df["width"], image_df["height"]]
    data = [d/IMAGE_SIZE for d in data]
    return data

def write_to_text_file(filename, image_data):
    with open(f'{filename}.txt', 'w') as f:
        for data in image_data:
            list_data = ' '.join([str(elem) for elem in data])
            f.write(list_data + "\n")

def generate_textfile_for_image(image_name):
    file_name = image_name.replace(".jpg", "")
    image_objects = df.loc[df["Image_ID"] == file_name]
    data = []
    for index, image_object in image_objects.iterrows():
        image_data = get_metric_data_from_image(image_object)
        labeled_name = classes[image_object["class"]]
        image_data.insert(0, labeled_name)
        data.append(image_data)

    write_to_text_file(f"{YOLO_DATA}/{file_name}", data)

In [ ]:
text_file_len = len(os.listdir(LABELS))
image_len = len(os.listdir(IMAGES))
assert(image_len == text_file_len)

In [ ]:
!python train.py --img 512 --batch 16 --epochs 10 --data dataset.yaml